# Лабораторна робота №4: Структури для роботи з великими обсягами даних в Python
### *виконала студентка групи ФБ-33 Журавльова Марія*

**Мета роботи:** отримати навички роботи із структурами для зберігання в Python (python, numpy, pandas, numpy array, dataframe, timeit)

## Перший рівень (спрощені завдання)
Наука про дані (Data science) дозволяє вирішувати широке коло задач, до яких належать не тільки суто наукові задачі теоретичного плану, але й цілком практичні, які є частиною нашого звичайного життя. Оскільки наразі доволі важливою є проблема економії енергоресурсів, то розглянемо задачу, пов’язану із споживанням електричної енергії домогосподарством.

Вашій увазі пропонується набір даних Individual household electric power consumption Data Set, який можна завантажити із UCI-репозиторію (за
посиланням Data folder): https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption
Детальний опис набору даних можна знайти за наведеним вище посиланням, якщо ж “коротко і по суті”, то це відомості щодо основних витрат електричної енергії домогосподарствами, зібрані впродовж 47 місяців (12.2006 – 11.2010).

***Перелік атрибутивної інформації:***
- date: дата виміру у форматі dd/mm/yyyy
- time: час у форматі hh:mm:ss
- global_active_power: активна потужність, яку споживає домогосподарство за хвилину (усереднено) [кВт]
- global_reactive_power: реактивна потужність, яку споживає домогосподарство за хвилину (усереднено) [кВт]
- voltage: напруга, усереднена за хвилину спостереження [В]
- global_intensity: усереднена силу струму для домогосподарства [A]
- sub_metering_1: набір споживачів енергії No1 [Вт-годин активної енергії], відповідає кухні, на якій є машина для миття посуду на мікрохвильовка (електричної плити немає, використовується газова).
- sub_metering_2: набір споживачів енергії No2 [Вт-годин активної енергії], відповіє пральні, в якій працює пральна машина, сушарка, холодильних та ввімкнено світло.
- sub_metering_3: набір споживачів енергії No3 [Вт-годин активної енергії], відповідає бойлеру та кондиціонеру.

#### Завдання першого рівня
Виконати всі завдання, використовуючи як numpy array, так і pandas dataframe, проаналізувати часові витрати на виконання процедур (профілювання часу виконання), зробити висновки щодо ситуацій, в яких має сенс віддати перевагу тій чи іншій структурі даних. Висновки оформитизвітом із зазначеним часом виконання та оцінкою по 5-бальній шкалі зручності виконання операцій відбору). Також варто звернути увагу на те, що дані, як і практично все в реальному
житті, можуть потребувати Вашої уваги - потрібно залишити лише ті спостереження, в яких немає порожніх спостережень (порожні значення – пусті поля між роздільником – ?).

In [3]:
import os
import urllib.request
import zipfile
import time
import pandas as pd
import numpy as np
from tabulate import tabulate
print("Setup Complete!")

Setup Complete!


Завантаження набору даних Individual household electric power consumption Data Set:

In [52]:
def clean_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Директорія {directory} створена.")
    
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.exists(filepath):
            os.unlink(filepath)
    print("Директорія очищена.")

def download_csv(directory):
    url = "https://archive.ics.uci.edu/static/public/235/individual+household+electric+power+consumption.zip"
    
    filename = 'Individual_Household_Electric_Power_Consumption.zip'
    filepath = os.path.join(directory, filename)
    if os.path.exists(filepath):
        print(f"Файл {filename} вже існує. Завантаження не потрібно.")
        return
    
    try:
        with urllib.request.urlopen(url) as wp:
            file_content = wp.read()
        
        with open(filepath, 'wb') as out_file:
            out_file.write(file_content)
        print(f"Файл {filename} завантажено успішно.")
    except urllib.error.URLError as e:
        print(f"Помилка завантаження файлу {filename}: {e}")

def unzip(directory):
    zip_file = os.path.join(directory, "Individual_Household_Electric_Power_Consumption.zip")
    
    if not os.path.exists(zip_file):
        print("Zip-файл не знайдено.")
        return
    
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(path=directory)
            print(f"Файли успішно розархівовано.")
            for filename in zip_ref.namelist():
                if filename.endswith('.txt'):
                    new_filename = 'individual_household_electric_power_consumption.csv'
                    os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
                    print(f"Файл перейменовано на: {new_filename}")
    except zipfile.BadZipFile as e:
        print(f"Помилка розпакування: {e}")

directory = r"C:/Users/User/vhi_venv/data_lab4"
clean_directory(directory)
download_csv(directory)
unzip(directory)

Директорія очищена.
Файл Individual_Household_Electric_Power_Consumption.zip завантажено успішно.
Файли успішно розархівовано.
Файл перейменовано на: individual_household_electric_power_consumption.csv


Читанна за допомогою ***pandas***.

In [4]:
def read_csv_pd(csv_file):
    try:
        df = pd.read_csv(csv_file, sep=';', na_values='?', low_memory=False)
        print("Файл завантажено успішно.")
        df_clean = df.dropna()
        return df_clean
    except Exception as e:
        print(f"Помилка при завантаженні файлу: {e}")
        return None

def print_table_pd(df):
    table_data = df.head(10).values.tolist()
    headers = df.columns.tolist()
    print("Перші 10 рядків DataFrame:")
    print(tabulate(table_data, headers=headers, tablefmt='pretty'))

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df = read_csv_pd(csv_file)
if df is not None:
    print_table_pd(df)

Файл завантажено успішно.
Перші 10 рядків DataFrame:
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
|    Date    |   Time   | Global_active_power | Global_reactive_power | Voltage | Global_intensity | Sub_metering_1 | Sub_metering_2 | Sub_metering_3 |
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
| 16/12/2006 | 17:24:00 |        4.216        |         0.418         | 234.84  |       18.4       |      0.0       |      1.0       |      17.0      |
| 16/12/2006 | 17:25:00 |        5.36         |         0.436         | 233.63  |       23.0       |      0.0       |      1.0       |      16.0      |
| 16/12/2006 | 17:26:00 |        5.374        |         0.498         | 233.29  |       23.0       |      0.0       |      2.0       |      17.0      |
| 16/12/2006 | 17:27:00 |        5.

Читання за допомогою ***numpy***

In [5]:
def read_csv_np(csv_file):
    types = [("Date", "U10"),
             ("Time", "U8"),
             ("Global_active_power", "float64"),
             ("Global_reactive_power", "float64"),
             ("Voltage", "float64"),
             ("Global_intensity", "float64"),
             ("Sub_metering_1", "float64"),
             ("Sub_metering_2", "float64"),
             ("Sub_metering_3", "float64")]

    try:
        df_np = np.genfromtxt(csv_file, missing_values=["?",np.nan], delimiter=';', dtype=types, encoding="UTF=8", names=True)
        print("Файл завантажено успішно.")
        
        mask = ~np.isnan(df_np['Global_active_power']) & \
               ~np.isnan(df_np['Global_reactive_power']) & \
               ~np.isnan(df_np['Voltage']) & \
               ~np.isnan(df_np['Global_intensity']) & \
               ~np.isnan(df_np['Sub_metering_1']) & \
               ~np.isnan(df_np['Sub_metering_2']) & \
               ~np.isnan(df_np['Sub_metering_3'])
        df_np_clean = df_np[mask]
        return df_np_clean
    except Exception as e:
        print(f"Помилка при завантаженні файлу: {e}")
        return None

def print_table_np(data):
    table_data = data[:10].tolist()
    print("Перші 10 рядків NumPy масиву:")
    print(tabulate(table_data, tablefmt='pretty'))

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df_np = read_csv_np(csv_file)

if df_np is not None:
    print("Імена стовпців:", df_np.dtype.names)
    print_table_np(df_np)

Файл завантажено успішно.
Імена стовпців: ('Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3')
Перші 10 рядків NumPy масиву:
+------------+----------+-------+-------+--------+------+-----+-----+------+
| 16/12/2006 | 17:24:00 | 4.216 | 0.418 | 234.84 | 18.4 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:25:00 | 5.36  | 0.436 | 233.63 | 23.0 | 0.0 | 1.0 | 16.0 |
| 16/12/2006 | 17:26:00 | 5.374 | 0.498 | 233.29 | 23.0 | 0.0 | 2.0 | 17.0 |
| 16/12/2006 | 17:27:00 | 5.388 | 0.502 | 233.74 | 23.0 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:28:00 | 3.666 | 0.528 | 235.68 | 15.8 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:29:00 | 3.52  | 0.522 | 235.02 | 15.0 | 0.0 | 2.0 | 17.0 |
| 16/12/2006 | 17:30:00 | 3.702 | 0.52  | 235.09 | 15.8 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:31:00 |  3.7  | 0.52  | 235.22 | 15.8 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:32:00 | 3.668 | 0.51  | 233.99 | 15.8 | 0.0 | 1.0 | 17.0 |
| 16/12/

1. Обрати всі записи, у яких загальна активна споживана потужність перевищує 5 кВт.

In [6]:
# Pandas
def filter_by_active_power_pd(df):
    if df is not None:
        df_filtered = df[df['Global_active_power'] > 5]
        return df_filtered
    else:
        print("Дані не завантажені!")
        return None

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df = read_csv_pd(csv_file)

start_time = time.time()
df_filtered = filter_by_active_power_pd(df)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_filtered is not None:
    print_table_pd(df_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 0.00997471809387207 секунд
Перші 10 рядків DataFrame:
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
|    Date    |   Time   | Global_active_power | Global_reactive_power | Voltage | Global_intensity | Sub_metering_1 | Sub_metering_2 | Sub_metering_3 |
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
| 16/12/2006 | 17:25:00 |        5.36         |         0.436         | 233.63  |       23.0       |      0.0       |      1.0       |      16.0      |
| 16/12/2006 | 17:26:00 |        5.374        |         0.498         | 233.29  |       23.0       |      0.0       |      2.0       |      17.0      |
| 16/12/2006 | 17:27:00 |        5.388        |         0.502         | 233.74  |       23.0       |      0.0       |      1.0       

In [7]:
# Numpy
def filter_by_active_power_np(df_np):
    if df_np is not None:
        mask = df_np['Global_active_power'] > 5
        df_np_filtered = df_np[mask]
        return df_np_filtered
    else:
        print("Дані не завантажені!")
        return None

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df_np = read_csv_np(csv_file)

start_time = time.time()
df_np_filtered = filter_by_active_power_np(df_np)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_np_filtered is not None:
    print_table_np(df_np_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 0.015019416809082031 секунд
Перші 10 рядків NumPy масиву:
+------------+----------+-------+-------+--------+------+-----+-----+------+
| 16/12/2006 | 17:25:00 | 5.36  | 0.436 | 233.63 | 23.0 | 0.0 | 1.0 | 16.0 |
| 16/12/2006 | 17:26:00 | 5.374 | 0.498 | 233.29 | 23.0 | 0.0 | 2.0 | 17.0 |
| 16/12/2006 | 17:27:00 | 5.388 | 0.502 | 233.74 | 23.0 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:35:00 | 5.412 | 0.47  | 232.78 | 23.2 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:36:00 | 5.224 | 0.478 | 232.99 | 22.4 | 0.0 | 1.0 | 16.0 |
| 16/12/2006 | 17:37:00 | 5.268 | 0.398 | 232.91 | 22.6 | 0.0 | 2.0 | 17.0 |
| 16/12/2006 | 17:44:00 | 5.894 |  0.0  | 232.69 | 25.4 | 0.0 | 0.0 | 16.0 |
| 16/12/2006 | 17:45:00 | 7.706 |  0.0  | 230.98 | 33.2 | 0.0 | 0.0 | 17.0 |
| 16/12/2006 | 17:46:00 | 7.026 |  0.0  | 232.21 | 30.6 | 0.0 | 0.0 | 16.0 |
| 16/12/2006 | 17:47:00 | 5.174 |  0.0  | 234.19 | 22.0 | 0.0 | 0.0 | 17.0 |
+------------+----------+-------+-------+-

2. Обрати всі записи, у яких вольтаж перевищую 235 В.

In [8]:
# Pandas
def filter_by_voltage_pd(df):
    if df is not None:
        df_filtered = df[df['Voltage'] > 235]
        return df_filtered
    else:
        print("Дані не завантажені!")
        return 

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df = read_csv_pd(csv_file)

start_time = time.time()
df_filtered = filter_by_voltage_pd(df)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_filtered is not None:
    print_table_pd(df_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 0.07144808769226074 секунд
Перші 10 рядків DataFrame:
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
|    Date    |   Time   | Global_active_power | Global_reactive_power | Voltage | Global_intensity | Sub_metering_1 | Sub_metering_2 | Sub_metering_3 |
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
| 16/12/2006 | 17:28:00 |        3.666        |         0.528         | 235.68  |       15.8       |      0.0       |      1.0       |      17.0      |
| 16/12/2006 | 17:29:00 |        3.52         |         0.522         | 235.02  |       15.0       |      0.0       |      2.0       |      17.0      |
| 16/12/2006 | 17:30:00 |        3.702        |         0.52          | 235.09  |       15.8       |      0.0       |      1.0       

In [9]:
# Numpy
def filter_by_voltage_np(df_np):
    if df_np is not None:
        mask = df_np['Voltage'] > 235
        df_np_filtered = df_np[mask]
        return df_np_filtered
    else:
        print("Дані не завантажені!")
        return None

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df_np = read_csv_np(csv_file)

start_time = time.time()
df_np_filtered = filter_by_voltage_np(df_np)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_np_filtered is not None:
    print_table_np(df_np_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 0.09390497207641602 секунд
Перші 10 рядків NumPy масиву:
+------------+----------+-------+-------+--------+------+-----+-----+------+
| 16/12/2006 | 17:28:00 | 3.666 | 0.528 | 235.68 | 15.8 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:29:00 | 3.52  | 0.522 | 235.02 | 15.0 | 0.0 | 2.0 | 17.0 |
| 16/12/2006 | 17:30:00 | 3.702 | 0.52  | 235.09 | 15.8 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:31:00 |  3.7  | 0.52  | 235.22 | 15.8 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:38:00 | 4.054 | 0.422 | 235.24 | 17.6 | 0.0 | 1.0 | 17.0 |
| 16/12/2006 | 17:39:00 | 3.384 | 0.282 | 237.14 | 14.2 | 0.0 | 0.0 | 17.0 |
| 16/12/2006 | 17:40:00 | 3.27  | 0.152 | 236.73 | 13.8 | 0.0 | 0.0 | 17.0 |
| 16/12/2006 | 17:41:00 | 3.43  | 0.156 | 237.06 | 14.4 | 0.0 | 0.0 | 17.0 |
| 16/12/2006 | 17:42:00 | 3.266 |  0.0  | 237.13 | 13.8 | 0.0 | 0.0 | 18.0 |
| 16/12/2006 | 17:43:00 | 3.728 |  0.0  | 235.84 | 16.4 | 0.0 | 0.0 | 17.0 |
+------------+----------+-------+-------+--

3. Обрати всі записи, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [10]:
# Pandas
def filter_by_current_and_consumption_pd(df):
    if df is not None:
        # Фільтруємо записи, де сила струму в межах 19-20 А
        df_filtered_current = df[(df['Global_intensity'] >= 19) & (df['Global_intensity'] <= 20)]
        
        # Пральна машина та холодильник споживають більше, ніж бойлер та кондиціонер
        df_filtered_consumption = df_filtered_current[df_filtered_current['Sub_metering_2'] > df_filtered_current['Sub_metering_3']]
        
        return df_filtered_consumption
    else:
        print("Дані не завантажені!")
        return None

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df = read_csv_pd(csv_file)

start_time = time.time()
df_filtered = filter_by_current_and_consumption_pd(df)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_filtered is not None:
    print_table_pd(df_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 0.02057337760925293 секунд
Перші 10 рядків DataFrame:
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
|    Date    |   Time   | Global_active_power | Global_reactive_power | Voltage | Global_intensity | Sub_metering_1 | Sub_metering_2 | Sub_metering_3 |
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
| 16/12/2006 | 18:09:00 |        4.464        |         0.136         | 234.66  |       19.0       |      0.0       |      37.0      |      16.0      |
| 17/12/2006 | 01:04:00 |        4.582        |         0.258         | 238.08  |       19.6       |      0.0       |      13.0      |      0.0       |
| 17/12/2006 | 01:08:00 |        4.618        |         0.104         | 239.61  |       19.6       |      0.0       |      27.0      

In [111]:
# Numpy
def filter_by_current_and_consumption_np(df_np):
    if df_np is not None:
        # Фільтруємо записи, де сила струму в межах 19-20 А
        mask_current = (df_np['Global_intensity'] >= 19) & (df_np['Global_intensity'] <= 20)
        
        # Пральна машина та холодильник споживають більше, ніж бойлер та кондиціонер
        mask_consumption = df_np['Sub_metering_2'] > df_np['Sub_metering_3']
        
        combined_mask = mask_current & mask_consumption
        df_np_filtered = df_np[combined_mask]
        return df_np_filtered
    else:
        print("Дані не завантажені!")
        return None

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df_np = read_csv_np(csv_file)

start_time = time.time()
df_np_filtered = filter_by_current_and_consumption_np(df_np)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_np_filtered is not None:
    print_table_np(df_np_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 0.05319786071777344 секунд
Перші 10 рядків NumPy масиву:
+------------+----------+-------+-------+--------+------+-----+------+------+
| 16/12/2006 | 18:09:00 | 4.464 | 0.136 | 234.66 | 19.0 | 0.0 | 37.0 | 16.0 |
| 17/12/2006 | 01:04:00 | 4.582 | 0.258 | 238.08 | 19.6 | 0.0 | 13.0 | 0.0  |
| 17/12/2006 | 01:08:00 | 4.618 | 0.104 | 239.61 | 19.6 | 0.0 | 27.0 | 0.0  |
| 17/12/2006 | 01:19:00 | 4.636 | 0.14  | 237.37 | 19.4 | 0.0 | 36.0 | 0.0  |
| 17/12/2006 | 01:20:00 | 4.634 | 0.152 | 237.17 | 19.4 | 0.0 | 35.0 | 0.0  |
| 17/12/2006 | 01:21:00 | 4.652 | 0.142 | 237.92 | 19.4 | 0.0 | 36.0 | 0.0  |
| 17/12/2006 | 01:52:00 | 4.622 | 0.24  | 239.59 | 19.2 | 0.0 | 37.0 | 0.0  |
| 17/12/2006 | 09:08:00 | 4.762 | 0.088 | 237.44 | 20.0 | 0.0 | 38.0 | 0.0  |
| 17/12/2006 | 09:09:00 | 4.506 | 0.088 | 237.25 | 19.0 | 0.0 | 38.0 | 0.0  |
| 17/12/2006 | 10:55:00 | 4.444 | 0.136 | 235.97 | 19.2 | 2.0 | 50.0 | 17.0 |
+------------+----------+-------

4. Обрати випадковим чином 500000 записів (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії

In [112]:
# Pandas
def random_sampling_and_avg_pd(df, sample_size=500000):
    if df is not None:
        random_samples = df.sample(n=sample_size, replace=False)

        avg_sub_metering_1 = random_samples['Sub_metering_1'].mean()
        avg_sub_metering_2 = random_samples['Sub_metering_2'].mean()
        avg_sub_metering_3 = random_samples['Sub_metering_3'].mean()
        
        print(f"Середнє споживання для Sub_metering_1: {avg_sub_metering_1}")
        print(f"Середнє споживання для Sub_metering_2: {avg_sub_metering_2}")
        print(f"Середнє споживання для Sub_metering_3: {avg_sub_metering_3}")
    else:
        print("Дані не завантажені!")

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df = read_csv_pd(csv_file)

start_time = time.time()
random_sampling_and_avg_pd(df)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

Файл завантажено успішно.
Середнє споживання для Sub_metering_1: 1.128392
Середнє споживання для Sub_metering_2: 1.303294
Середнє споживання для Sub_metering_3: 6.469038

Час виконання фільтрації: 0.09558320045471191 секунд


In [113]:
# Numpy
def random_sampling_and_avg_np(df_np, sample_size=500000):
    if df_np is not None:
        random_indices = np.random.choice(df_np.shape[0], size=sample_size, replace=False)
        
        random_samples = df_np[random_indices]
        
        avg_sub_metering_1 = np.mean(random_samples['Sub_metering_1'])
        avg_sub_metering_2 = np.mean(random_samples['Sub_metering_2'])
        avg_sub_metering_3 = np.mean(random_samples['Sub_metering_3'])
        
        print(f"Середнє споживання для Sub_metering_1: {avg_sub_metering_1}")
        print(f"Середнє споживання для Sub_metering_2: {avg_sub_metering_2}")
        print(f"Середнє споживання для Sub_metering_3: {avg_sub_metering_3}")
    else:
        print("Дані не завантажені!")

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df_np = read_csv_np(csv_file)

start_time = time.time()
random_sampling_and_avg_np(df_np)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

Файл завантажено успішно.
Середнє споживання для Sub_metering_1: 1.118056
Середнє споживання для Sub_metering_2: 1.302092
Середнє споживання для Sub_metering_3: 6.453364

Час виконання фільтрації: 0.17026042938232422 секунд


5. Обрати ті записи, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [124]:
# Pandas
def filter_by_time_and_consumption_pd(df):
    if df is not None:
        df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
        
        df_filtered = df[
            (df['Time'] > pd.to_datetime('18:00:00', format='%H:%M:%S').time()) &
            (df['Global_active_power'] > 6) &
            (df['Sub_metering_2'] > df['Sub_metering_1']) &
            (df['Sub_metering_2'] > df['Sub_metering_3'])]
        
        mid_point = len(df_filtered) // 2
        first_half = df_filtered[:mid_point]
        second_half = df_filtered[mid_point:]
        
        first_half_sample = first_half.iloc[::3]
        second_half_sample = second_half.iloc[::4]
        
        final_sample = pd.concat([first_half_sample, second_half_sample])
        
        return final_sample
    else:
        print("Дані не завантажені!")
        return None

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df = read_csv_pd(csv_file)

start_time = time.time()
df_filtered = filter_by_time_and_consumption_pd(df)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_filtered is not None:
    print_table_pd(df_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 3.214022159576416 секунд
Перші 10 рядків DataFrame:
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
|    Date    |   Time   | Global_active_power | Global_reactive_power | Voltage | Global_intensity | Sub_metering_1 | Sub_metering_2 | Sub_metering_3 |
+------------+----------+---------------------+-----------------------+---------+------------------+----------------+----------------+----------------+
| 16/12/2006 | 18:05:00 |        6.052        |         0.192         | 232.93  |       26.2       |      0.0       |      37.0      |      17.0      |
| 16/12/2006 | 18:08:00 |        6.308        |         0.116         | 232.25  |       27.0       |      0.0       |      36.0      |      17.0      |
| 28/12/2006 | 20:58:00 |        6.386        |         0.374         | 236.63  |       27.0       |      1.0       |      36.0      | 

In [11]:
def filter_by_time_and_consumption_np(df_np):
    if df_np is not None:
        times = np.array([np.datetime64(f"2025-01-01T{t}") for t in df_np['Time']])
        threshold_time = np.datetime64("2025-01-01T18:00:00")
        mask_time = times > threshold_time
        mask_power = df_np['Global_active_power'] > 6
        mask = mask_time & mask_power
        df_filtered_time_power = df_np[mask]
        
        mask_consumption = df_filtered_time_power['Sub_metering_2'] > np.maximum(df_filtered_time_power['Sub_metering_1'], df_filtered_time_power['Sub_metering_3'])
        df_filtered_consumption = df_filtered_time_power[mask_consumption]
        
        mid_point = len(df_filtered_consumption) // 2
        first_half = df_filtered_consumption[:mid_point]
        second_half = df_filtered_consumption[mid_point:]
        
        first_half_sample = first_half[::3]
        second_half_sample = second_half[::4]
        
        final_sample = np.concatenate((first_half_sample, second_half_sample))
        
        return final_sample
    else:
        print("Дані не завантажені!")
        return None

csv_file = r"C:\Users\User\vhi_venv\data_lab4\individual_household_electric_power_consumption.csv"
df_np = read_csv_np(csv_file)

start_time = time.time()
df_np_filtered = filter_by_time_and_consumption_np(df_np)
end_time = time.time()
execution_time = end_time - start_time
print(f"\nЧас виконання фільтрації: {execution_time} секунд")

if df_np_filtered is not None:
    print_table_np(df_np_filtered)

Файл завантажено успішно.

Час виконання фільтрації: 1.6416399478912354 секунд
Перші 10 рядків NumPy масиву:
+------------+----------+-------+-------+--------+------+------+------+------+
| 16/12/2006 | 18:05:00 | 6.052 | 0.192 | 232.93 | 26.2 | 0.0  | 37.0 | 17.0 |
| 16/12/2006 | 18:08:00 | 6.308 | 0.116 | 232.25 | 27.0 | 0.0  | 36.0 | 17.0 |
| 28/12/2006 | 20:58:00 | 6.386 | 0.374 | 236.63 | 27.0 | 1.0  | 36.0 | 17.0 |
| 28/12/2006 | 21:02:00 | 8.088 | 0.262 | 235.5  | 34.4 | 1.0  | 72.0 | 17.0 |
| 28/12/2006 | 21:05:00 | 7.23  | 0.152 | 235.22 | 30.6 | 1.0  | 73.0 | 17.0 |
| 28/12/2006 | 21:08:00 | 7.352 |  0.0  | 235.45 | 31.2 | 1.0  | 73.0 | 17.0 |
| 28/12/2006 | 21:11:00 | 9.048 |  0.0  | 231.48 | 39.0 | 34.0 | 71.0 | 16.0 |
| 28/12/2006 | 21:14:00 | 9.118 | 0.108 | 231.18 | 39.4 | 36.0 | 72.0 | 16.0 |
| 28/12/2006 | 21:17:00 | 7.04  | 0.13  | 233.27 | 30.2 | 37.0 | 38.0 | 17.0 |
| 29/12/2006 | 21:16:00 | 6.146 | 0.116 | 230.53 | 26.6 | 0.0  | 70.0 | 0.0  |
+------------+--------

**Висновок:** 
|        | Завдання 1 | Завдання 2 | Завдання 3 | Завдання 4 | Завдання 5 |
|--------|------------|------------|------------|------------|------------|
| pandas dataframe | 0.00997471809387207 | 0.07144808769226074  | 0.02057337760925293 | 0.09558320045471191 | 3.214022159576416 |
| numpy  | 0.015019416809082031  | 0.09390497207641602 | 0.05319786071777344 | 0.17026042938232422 | 1.6416399478912354 |

*Pandas DataFrame*: зручність - 5, швидкість - 5.

*Numpy*: зручність - 4, швидкість - 4.